In [ ]:
# 데이터셋 생성을 위해 이미지 크롭하는 코드

In [4]:
import os
import json
from PIL import Image

# 경로 설정
a_path = '/data_2/ace_minjae/yolo_project/dataset/for_crop_image'  # 이미지 경로
b_path = '/data_2/ace_minjae/yolo_project/dataset/for_crop_label'  # json 경로
c_path = '/data_2/ace_minjae/yolo_project/crop_dataset'  # 새로운 폴더 경로

# 모든 폴더와 파일을 순회
for folder_name in os.listdir(a_path):
    folder_path = os.path.join(a_path, folder_name)
    if os.path.isdir(folder_path):
        # /c에 새로운 폴더 생성
        c_folder_path = os.path.join(c_path, folder_name)
        os.makedirs(c_folder_path, exist_ok=True)
        
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.png'):
                png_file_path = os.path.join(folder_path, file_name)
                json_folder_path = os.path.join(b_path, folder_name + '_json')
                json_file_path = os.path.join(json_folder_path, file_name.replace('.png', '.json'))
                
                if os.path.exists(json_file_path):
                    with open(json_file_path, 'r') as f:
                        data = json.load(f)
                        
                        # 이미지 파일 이름을 통해 이미지 id 찾기
                        image_id = None
                        for image in data['images']:
                            if image['file_name'] == file_name:
                                image_id = image['id']
                                break
                        
                        if image_id is not None:
                            # bounding box 정보 추출
                            for annotation in data['annotations']:
                                if annotation['image_id'] == image_id:
                                    bbox = annotation['bbox']
                                    x, y, width, height = bbox
                                    # 이미지 열기
                                    with Image.open(png_file_path) as img:
                                        # 이미지 crop
                                        cropped_img = img.crop((x, y, x + width, y + height))
                                        # crop된 이미지 저장
                                        cropped_img.save(os.path.join(c_folder_path, file_name))
                        else:
                            print(f"Image ID not found for {file_name} in {json_file_path}")
                else:
                    print(f"JSON file not found for {png_file_path}")
